In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import json
import re

In [ ]:
# Step 1: Parse initial data from Wikipedia's highest-grossing films list
url = "https://en.wikipedia.org/wiki/List_of_highest-grossing_films"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find the first table (highest-grossing films)
table = soup.find('table', {'class': 'wikitable'})

# Initialize lists to store data
titles = []
years = []
revenues = []
directors = []
countries = []

# Parse table rows
for row in table.find_all('tr')[1:]:  # Skip header row
    cols = row.find_all(['td', 'th'])
    if len(cols) >= 4:  # Ensure row has enough columns
        # Extract title and get the film's Wikipedia link
        title_cell = cols[2].find('i')
        if title_cell:
            title = title_cell.text.strip()
            titles.append(title)
            
            # Extract year
            year = cols[4].text.strip()
            try:
                year = int(year)
            except ValueError:
                try:
                    year = int(year.split('-')[0])
                except:
                    year = None
            years.append(year)
            
            # Extract revenue
            revenue = cols[3].text.strip()
            revenue = revenue.split('(')[0].strip()
            revenues.append(revenue)

            # Get the film's Wikipedia page link
            film_link = title_cell.find('a')
            if film_link:
                film_href = "https://en.wikipedia.org" + film_link.get('href')
                # Fetch the film's page
                film_response = requests.get(film_href)
                film_soup = BeautifulSoup(film_response.content, 'html.parser')
                
                # Find director(s)
                director_row = film_soup.find('th', string=re.compile('Directed by'))
                if director_row:
                    director_data = director_row.find_next_sibling('td')
                    film_directors = []
                    if director_data:
                        # Extract all director names
                        for director in director_data.stripped_strings:
                            if director not in ['[', ']', ',']:
                                film_directors.append(director)
                    directors.append(film_directors)
                else:
                    directors.append([])
                
                # Find country
                country_row = film_soup.find('th', string=re.compile('Countr(y|ies)'))
                if country_row:
                    country_data = country_row.find_next_sibling('td')
                    if country_data:
                        # Handle both list and plain text formats
                        country_list = country_data.find('ul')
                        if country_list:
                            # Get first country from list items
                            first_li = country_list.find('li')
                            if first_li:
                                # Remove reference citations and clean up
                                film_country = re.sub(r'\[\d+\]', '', first_li.text).strip()
                        else:
                            # Handle plain text format
                            film_country = re.sub(r'\[\d+\]', '', country_data.text.strip().split(',')[0].split('\n')[0]).strip()
                    else:
                        film_country = 'Unknown'
                    countries.append(film_country)
                else:
                    countries.append('Unknown')

In [ ]:
# Create DataFrame
df = pd.DataFrame({
    'title': titles,
    'release_year': years,
    'box_office': revenues,
    'directors': directors,
    'country': countries
})

# Create SQLite database
conn = sqlite3.connect('films.db')
cursor = conn.cursor()

# Create films table
cursor.execute('''
CREATE TABLE IF NOT EXISTS films (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    release_year INTEGER,
    box_office TEXT,
    director TEXT,
    country TEXT
)
''')

# Insert data into database
for index, row in df.iterrows():
    cursor.execute('''
    INSERT INTO films (title, release_year, box_office, director, country)
    VALUES (?, ?, ?, ?, ?)
    ''', (row['title'], row['release_year'], row['box_office'], json.dumps(row['directors']), row['country']))

conn.commit()

In [ ]:
# Export to JSON
df.to_json('films.json', orient='records', indent=4)

# Close database connection
conn.close()

print("Data successfully extracted, saved to database and exported to JSON!")